In [1]:
# Import libraries
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# This is a magic command. It will display the plotting image directly below the code cell
%matplotlib inline  

In [2]:
from scipy.io import loadmat  # for loading MATLAB files

dataFilePath = os.getcwd() + 'Data/ex3data1.mat'
data = loadmat(dataFilePath)

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
    #*************************************************************************#
    #                 EXTRACTION OF TRAINING DATA: (X,Y)                      #
    #                                                                         #
    #*************************************************************************#

# preparing the training input matrix
X  = data['X']                           # (5000,400)-dim array of all training inputs

n  = data['X'].shape[1]                  # input layer size = 400
m  = data['X'].shape[0]                  # total number of training examples

I  = np.ones((m,1))                      # (5000,1)-dim array of ones

A1 = np.c_[I, X]                         # (5000,401)-dim array   ### A^(1)
A1 = np.matrix(A1)      #numpy matrix    # A^(1) 

y  = data['y']                           # (5000,1)-dim array of all training outputs                     
y  = np.matrix(y)       #numpy matrix    # labels 10,1,...,9 in batches of 500 => (5000,1)-dim                           


# preparing the training output matrix
encoder = OneHotEncoder(sparse=False)
Y = encoder.fit_transform(y)
Y = np.matrix(Y)                         # (5000,10)-dim matrix of all training outputs

In [5]:
    #*************************************************************************#
    #                   PARAMETERS OF THE NEURAL NETWORK                      #
    #                                                                         #
    #*************************************************************************#

L = 25                                   # Hidden Layer Size
K = 10                                   # Number of Labels (DO NOT CHANGE THIS unless necessary)
reg = 1                                  # Regularization parameter
eta = 0.001                              # Learning rate
epochs = 100                             # Number of iterations

In [6]:
def randomlyInitializeWeights(l_in, l_out):
    epsilon_init = 0.12
    weights = np.random.rand(l_out, 1 + l_in) * 2 * epsilon_init - epsilon_init
    weights = np.matrix(weights)

    return weights

In [7]:
theta1 = randomlyInitializeWeights(n,L)
theta2 = randomlyInitializeWeights(L,K)

#*************************************************************************#
# In order to use scipy.optimize, we need to use a 1D array as a variable #
# and define a cost function that depends only on that array.             #

weights = np.array(np.hstack((theta1.flatten(), theta2.flatten())))[0]
#*************************************************************************#

In [8]:
def sigmoid(z):
    g = 1 / (1 + np.exp(-z))
    return g

In [9]:
def FeedForwardAll(theta1, theta2):
    
    #*************************************************************************#
    # This will feedforward all of the training data at once.                 #
    # For individual predictions, check the rows of A3.                       #
    #*************************************************************************#
    
    Z2 = A1 * theta1.T
    G2 = sigmoid(Z2)                   # needed later for calculation of D2
    A2 = np.hstack((I, G2))
    
    Z3 = A2 * theta2.T
    A3 = sigmoid(Z3)
    
    return G2, A2, A3

In [10]:
def BackPropagation(theta1, theta2):
    
    G2, A2, A3 = FeedForwardAll(theta1, theta2) 
    
    #*********************************Error Terms : Start*************************************** 
        
    theta2_nobias = theta2[:, 1:]  # removes the first column containing the biases
    
    # (5000,10)-dim matrix of all 10-dim error terms at output:
    
    D3 = A3 - Y
    
    # (5000,25)-dim matrix of all 25-dim error terms at the hidden layer:
    
    D2 = np.multiply(np.multiply(G2,(1-G2)), (D3 * theta2_nobias)) 
    
    #**********************************Error Terms : End**************************************** 
    
    
    # Compute the gradients of the regularized cost function
    
    grad1 = (1/m) * (D2.T * A1) + ((reg/m) * theta1)
    grad2 = (1/m) * (D3.T * A2) + ((reg/m) * theta2)
    
    return grad1, grad2

In [11]:
# definition of the unregularized cost

def cost(A3, Y):
    J = -(1/m) * np.trace( Y * np.log(A3.T) + (1-Y) * np.log(1-A3.T) )
    return J

# definition of the regularized cost

def regularizedcost(weights):
    
    # reconstructing theta1 and theta2
    theta1 = np.matrix(weights[:L*(1+n)].reshape(L,1+n))
    theta2 = np.matrix(weights[L*(1+n):].reshape(K,1+L))
    
    A3 = FeedForwardAll(theta1, theta2)[2]
    
    phi1 = theta1.T * theta1
    phi2 = theta2.T * theta2
    regularization = (reg/(2 * m)) * (np.trace(phi1) + np.trace(phi2) - phi1[0,0] - phi2[0,0])
    
    J = cost(A3,Y) + regularization
    
    return J

In [12]:
def gradcost(weights):
    
    # reconstructing theta1 and theta2
    theta1 = np.matrix(weights[:L*(1+n)].reshape(L,1+n))
    theta2 = np.matrix(weights[L*(1+n):].reshape(K,1+L))
    
    grad1, grad2 = BackPropagation(theta1, theta2)
    
    # like before, we need to stack them up in a 1D array for np.optimize
    lingrad = np.array(np.hstack((grad1.flatten(), grad2.flatten())))[0]
    
    return lingrad

In [13]:
from scipy.optimize import fmin_cg as find_bottom_of

    #*************************************************************************#
    # We need to optimize the regularized cost function in order to find the  #
    # best weights. Since there is no guarantee that there is a global bottom #
    # of the cost function, we need to limit the maximum number of iterations #
    # to the specified number of epochs.                                      #
    #                                                                         #
    # If you wish to remove the regularization, just set the regularization   #
    # parameter ("reg" above) to zero. Lastly, the learning rate decides the  #
    # size of the steps taken to slide to the bottom in every iteration.      #
    #*************************************************************************#

trained_weights = find_bottom_of(f=regularizedcost, x0=weights, fprime=gradcost, epsilon=eta, maxiter=epochs)

trained_theta1 = np.matrix(trained_weights[:L*(1+n)].reshape(L,1+n))
trained_theta2 = np.matrix(trained_weights[L*(1+n):].reshape(K,1+L))

         Current function value: 0.352896
         Iterations: 100
         Function evaluations: 227
         Gradient evaluations: 227


In [14]:
    #*************************************************************************#
    #                     Performance on the Training Data                    #
    #                                                                         #
    #*************************************************************************#
    
A3 = FeedForwardAll(trained_theta1, trained_theta2)[2]

# converting the rows of A3 into lists of mostly zeroes,
# and a single value of "1" where its most large.

for i in range(m): A3[i,np.argmax(A3[i])] = 1
pred = (A3 == np.ones((m,K))).astype(float)

# checking if the predictions are correct and counting 
# the total number of correct predictions
correct = 0
for i in range(m): 
        if np.array_equal(Y[i],pred[i]): correct += 1

print("Epoch complete")
print("Cost reduced to: ", regularizedcost(trained_weights))
print("Training Accuracy = ", correct*100/m,"%") 

Epoch complete
Cost reduced to:  0.3528960103692646
Training Accuracy =  98.6 %
